In [42]:
# Simple smo algorithm for SVM. 
# The results of each running may be different due to random selection inside simple smo algorithm.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time 
from sklearn.preprocessing import LabelBinarizer, StandardScaler


def accuracy(ypred, yreal):
    return np.sum(ypred==yreal)/float(len(yreal))

def scaler(X, mean, std):
    """
        mean and std are two row vectors. For each row of X, it substract A and divide the resulting row by B elementwise. 
    """
    # In case std has zeros. Replace zeros by ones. 
    A = np.zeros((1, X.shape[1]))
    for i in np.arange(X.shape[1]):
        if std[0, i] == 0:
            A[0, i] = 1
        else:
            A[0, i] = std[0, i]
    return (X - np.ones((X.shape[0],1))@mean)/(np.ones((X.shape[0],1))@A)

class SVM():
    """
        simple smo
    """
    def __init__(self, iterations=500, kernel="poly", C=1.0, tol = 0.001, gamma = "auto", degree = 3, coef0=0.0):
        self.kernels = {
            "linear" : self.linear,
            "poly" : self.poly,
            "rbf" : self.rbf,
            "sigmoid" : self.sigmoid
        }
        self.iterations = iterations
        self.kernel = self.kernels[kernel]
        self.C = C
        self.tol = tol
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0

    def fit(self, X, y):
        """
            X: dataset without extended column of ones 
            y: target or label.
        """
        m = X.shape[0]
        if m != y.shape[0]:
            print("Error of simple_smo: Dimension of data and target don't match.")
            return None
        if self.gamma == "auto":
            self.gamma = 1/X.shape[1] # This is helpful for rbf and sigmoid.

        KerMat = self.kernel(X, X) # All product of xi and xj
        alphas = np.zeros(m)
        b = 0
        passes = 0
        while (passes < self.iterations):
            num_changed_alphas = 0
            for i in np.arange(m):
                Ei = np.dot(KerMat[i,:], alphas*y) + b - y[i]
                if (y[i]*Ei < - self.tol and alphas[i] < self.C) or (y[i]*Ei > self.tol and alphas[i] > 0):
                    j = self.select_j(i, m)
                    Ej = np.dot(KerMat[j,:], alphas*y) + b - y[j]
                    alphaIold = np.copy(alphas[i]); # prevent alphaIold being modified unexpectedly
                    alphaJold = np.copy(alphas[j])
                    if (y[i] != y[j]):
                        L = max(0, alphas[j] - alphas[i])
                        H = min(self.C, self.C + alphas[j] - alphas[i])
                    else:
                        L = max(0, alphas[j] + alphas[i] - self.C)
                        H = min(self.C, alphas[j] + alphas[i])
                    if L==H:
                        continue # continue to next i
                    eta = 2.0 * KerMat[i,j] - KerMat[i,i] - KerMat[j,j]
                    if eta >= 0:
                        continue
                    # Compute and clip new value for alphas[j]
                    alphas[j] = alphas[j] - y[j]*(Ei - Ej)/eta  
                    alphas[j] = self.clip(alphas[j], L, H)
                    if abs(alphaJold - alphas[j]) < 1e-5:
                        continue 
                    alphas[i] = alphas[i] + y[i]*y[j]*(alphaJold - alphas[j])
                    # compute b1 and b2
                    b1 = b - Ei- y[i]*(alphas[i]-alphaIold)*KerMat[i,i] - y[j]*(alphas[j]-alphaJold)*KerMat[i,j]
                    b2 = b - Ej- y[i]*(alphas[i]-alphaIold)*KerMat[i,j] - y[j]*(alphas[j]-alphaJold)*KerMat[j,j]
                    # compute b
                    if 0 < alphas[i] and alphas[i] < self.C:
                        b = b1
                    elif 0 < alphas[j] and alphas[j] < self.C:
                        b = b2
                    else:
                        b = (b1 + b2)/2.0  
                    num_changed_alphas += 1
                    # end if
            # end for
            if (num_changed_alphas == 0): 
                passes += 1
            else: 
                passes = 0
        self.b = b
        self.SV = X[alphas != 0] # support vectors
        self.SValphas = alphas[alphas != 0]
        self.SVlabels = y[alphas != 0]

    def clip(self, alpha, L, H):
        ''' 
            clip alpha to lie within range [L, H].
        '''
        if alpha < L:
            return L
        elif alpha > H:
            return H
        else:
            return alpha
    
    def select_j(self, i, m):
        """
            Select a number in np.arange(m) that doesn't equal to i.
        """
        j = np.random.choice(m-1, 1)
        if j == i:
            j = j + 1
        return j

    def predict(self, Xtest, prob = False):
        ypred = np.dot(self.kernel(Xtest, self.SV), self.SValphas*self.SVlabels) + self.b
        # end calculation of f
        if prob == True:
            return ypred
        else:
            ypred[ypred>1e-5]=1
            ypred[ypred<=1e-5]=-1
        return ypred

    def linear(self, u, v):
        """
            u, v might both be 2d array with the same shape[1].
        """
        return np.dot(u, v.T)

    def poly(self, u, v):
        """
            polynomial kernel (coef0 + \gamma u \cdot v)^d where d is degree. u and v are 1d vectors.
            u, v might both be 2d array with the same shape[1].
        """
        return (self.coef0 + self.gamma * np.dot(u, v.T))**self.degree

    def rbf(self, u, v):
        """
            u, v might both be 2d array with the same shape[1].
        """
        if u.ndim == 1 and v.ndim == 1:
            return np.exp(-self.gamma * np.sum((u-v)**2))
        elif (u.ndim ==1 and v.ndim != 1):
            return np.exp(-self.gamma * np.sum((u-v)**2, axis=1))
        elif (u.ndim!=1 and v.ndim ==1):
            return np.exp(-self.gamma * np.sum((u-v)**2, axis=1))       
        else: 
            res = np.zeros((u.shape[0], v.shape[0]))
            for i in range(u.shape[0]):
                res[i] = np.exp(-self.gamma * np.sum((u[i]-v)**2, axis=1))   
            return res

    def sigmoid(self, u, v):
        """
            u, v might both be 2d array with the same shape[1].
        """
        return np.tanh(self.gamma * np.dot(u, v.T) + self.coef0)
    
class normalizer():
    def __init__(self):
        self.mean = 0
        self.std = 0

    def fit(self, X):
        self.mean = np.mean(X, axis=0) # mean of each column vector
        self.std = np.std(X, axis=0) # std of each column vector
        self.std[self.std <= 1e-5] = 1

    def transform(self, X):
        """
            feature normalization. Each row of X represents a point in R^d. 
            Substract by the mean of X and then divided by the std of X.
        """
        return (X - self.mean)/self.std

Xtrain = pd.read_csv("MNIST_X_train.csv").values
ytrain = pd.read_csv("MNIST_Y_train.csv").values
Xtest = pd.read_csv("MNIST_X_test.csv").values
ytest = pd.read_csv("MNIST_Y_test.csv").values

print("The shape of Xtrain is {}".format(Xtrain.shape))
print("The shape of ytrain is {}".format(ytrain.shape))
print("The shape of Xtest is {}".format(Xtest.shape))
print("The shape of ytest is {}".format(ytest.shape))

ytrain, ytest = ytrain.flatten(), ytest.flatten()

lb = LabelBinarizer(neg_label=-1)
lb.fit(ytrain)
ytrain_ohe = lb.transform(ytrain)
ytest_ohe  = lb.transform(ytest)

# Feature scaling
scaler = normalizer()
scaler.fit(Xtrain)
normalized_Xtrain = scaler.transform(Xtrain)
normalized_Xtest = scaler.transform(Xtest)

The shape of Xtrain is (2000, 784)
The shape of ytrain is (2000, 1)
The shape of Xtest is (500, 784)
The shape of ytest is (500, 1)


In [43]:
start = time.time()
# linear kernel
# one vs one approach
labels = np.zeros((Xtest.shape[0], 10))
for i in range(10):
    for j in range(10):
        if j > i:
            # prepare data and target for i vs j
            data = normalized_Xtrain[(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            target = ytrain_ohe[:,i][(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            # Train class i vs class j
            clf = SVM(kernel = "linear", iterations = 30)
            clf.fit(data, target)
            # compute training accuracy
            predLabels = clf.predict(data)
            score = accuracy(target, predLabels)
            print("Training class {} vs class {} is complete. The training accuracy is {:.2f}%".format(i,j,score*100))

            pred = clf.predict(normalized_Xtest)
            labels[:, i][pred==1] += 1
            labels[:, j][pred==-1] += 1
            
ypred = np.argmax(labels, axis=1)
end = time.time()

score = accuracy(ytest, ypred)
print("Using linear kernel, the accuracy of multiclass classification is {:.2f}%".format(score*100)) 
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs class 1 is complete. The training accuracy is 100.00%
Training class 0 vs class 2 is complete. The training accuracy is 100.00%
Training class 0 vs class 3 is complete. The training accuracy is 100.00%
Training class 0 vs class 4 is complete. The training accuracy is 100.00%
Training class 0 vs class 5 is complete. The training accuracy is 100.00%
Training class 0 vs class 6 is complete. The training accuracy is 100.00%
Training class 0 vs class 7 is complete. The training accuracy is 100.00%
Training class 0 vs class 8 is complete. The training accuracy is 100.00%
Training class 0 vs class 9 is complete. The training accuracy is 100.00%
Training class 1 vs class 2 is complete. The training accuracy is 100.00%
Training class 1 vs class 3 is complete. The training accuracy is 100.00%
Training class 1 vs class 4 is complete. The training accuracy is 100.00%
Training class 1 vs class 5 is complete. The training accuracy is 100.00%
Training class 1 vs class 6 is complet

In [44]:
start = time.time()
# poly kernel
# one vs one approach
labels = np.zeros((Xtest.shape[0], 10))
for i in range(10):
    for j in range(10):
        if j > i:
            # prepare data and target for i vs j
            data = normalized_Xtrain[(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            target = ytrain_ohe[:,i][(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            # Train class i vs class j
            clf = SVM(kernel = "poly", gamma = 1, iterations = 50, degree=5)
            clf.fit(data, target)
            # compute training accuracy
            predLabels = clf.predict(data)
            score = accuracy(target, predLabels)
            print("Training class {} vs class {} is complete. The training accuracy is {:.2f}%".format(i,j,score*100))

            pred = clf.predict(normalized_Xtest)
            labels[:, i][pred==1] += 1
            labels[:, j][pred==-1] += 1
            
ypred = np.argmax(labels, axis=1)
end = time.time()

score = accuracy(ytest, ypred)
print("Using poly kernel, the accuracy of multiclass classification is {:.2f}%".format(score*100)) 
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs class 1 is complete. The training accuracy is 100.00%
Training class 0 vs class 2 is complete. The training accuracy is 100.00%
Training class 0 vs class 3 is complete. The training accuracy is 100.00%
Training class 0 vs class 4 is complete. The training accuracy is 100.00%
Training class 0 vs class 5 is complete. The training accuracy is 100.00%
Training class 0 vs class 6 is complete. The training accuracy is 100.00%
Training class 0 vs class 7 is complete. The training accuracy is 100.00%
Training class 0 vs class 8 is complete. The training accuracy is 100.00%
Training class 0 vs class 9 is complete. The training accuracy is 100.00%
Training class 1 vs class 2 is complete. The training accuracy is 100.00%
Training class 1 vs class 3 is complete. The training accuracy is 100.00%
Training class 1 vs class 4 is complete. The training accuracy is 100.00%
Training class 1 vs class 5 is complete. The training accuracy is 100.00%
Training class 1 vs class 6 is complet

In [45]:
# rbf kernel
start = time.time()
# one vs one approach
labels = np.zeros((Xtest.shape[0], 10))
for i in range(10):
    for j in range(10):
        if j > i:
            # prepare data and target for i vs j
            data = normalized_Xtrain[(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            target = ytrain_ohe[:,i][(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            # Train class i vs class j
            clf = SVM(kernel = "rbf", iterations = 20)
            clf.fit(data, target)
            # compute training accuracy
            predLabels = clf.predict(data)
            score = accuracy(target, predLabels)
            print("Training class {} vs class {} is complete. The training accuracy is {:.2f}%".format(i,j,score*100))

            pred = clf.predict(normalized_Xtest)
            labels[:, i][pred==1] += 1
            labels[:, j][pred==-1] += 1
            
ypred = np.argmax(labels, axis=1)
end = time.time()

score = accuracy(ytest, ypred)
print("Using rbf kernel, the accuracy of multiclass classification is {:.2f}%".format(score*100)) 
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs class 1 is complete. The training accuracy is 100.00%
Training class 0 vs class 2 is complete. The training accuracy is 100.00%
Training class 0 vs class 3 is complete. The training accuracy is 100.00%
Training class 0 vs class 4 is complete. The training accuracy is 100.00%
Training class 0 vs class 5 is complete. The training accuracy is 99.74%
Training class 0 vs class 6 is complete. The training accuracy is 100.00%
Training class 0 vs class 7 is complete. The training accuracy is 100.00%
Training class 0 vs class 8 is complete. The training accuracy is 99.74%
Training class 0 vs class 9 is complete. The training accuracy is 100.00%
Training class 1 vs class 2 is complete. The training accuracy is 99.55%
Training class 1 vs class 3 is complete. The training accuracy is 98.86%
Training class 1 vs class 4 is complete. The training accuracy is 99.54%
Training class 1 vs class 5 is complete. The training accuracy is 99.76%
Training class 1 vs class 6 is complete. The

In [46]:
# sigmoid kernel
start = time.time()
# one vs one approach
labels = np.zeros((Xtest.shape[0], 10))
for i in range(10):
    for j in range(10):
        if j > i:
            # prepare data and target for i vs j
            data = normalized_Xtrain[(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            target = ytrain_ohe[:,i][(ytrain_ohe[:, i]==1)+(ytrain_ohe[:, j]==1)]
            # Train class i vs class j
            clf = SVM(kernel = "sigmoid", iterations = 20)
            clf.fit(data, target)
            # compute training accuracy
            predLabels = clf.predict(data)
            score = accuracy(target, predLabels)
            print("Training class {} vs class {} is complete. The training accuracy is {:.2f}%".format(i,j,score*100))

            pred = clf.predict(normalized_Xtest)
            labels[:, i][pred==1] += 1
            labels[:, j][pred==-1] += 1
            
ypred = np.argmax(labels, axis=1)
end = time.time()

score = accuracy(ytest, ypred)
print("Using sigmoid kernel, the accuracy of multiclass classification is {:.2f}%".format(score*100)) 
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs class 1 is complete. The training accuracy is 100.00%
Training class 0 vs class 2 is complete. The training accuracy is 98.10%
Training class 0 vs class 3 is complete. The training accuracy is 98.30%
Training class 0 vs class 4 is complete. The training accuracy is 99.75%
Training class 0 vs class 5 is complete. The training accuracy is 98.71%
Training class 0 vs class 6 is complete. The training accuracy is 99.00%
Training class 0 vs class 7 is complete. The training accuracy is 99.76%
Training class 0 vs class 8 is complete. The training accuracy is 98.94%
Training class 0 vs class 9 is complete. The training accuracy is 99.49%
Training class 1 vs class 2 is complete. The training accuracy is 98.89%
Training class 1 vs class 3 is complete. The training accuracy is 98.64%
Training class 1 vs class 4 is complete. The training accuracy is 99.77%
Training class 1 vs class 5 is complete. The training accuracy is 99.76%
Training class 1 vs class 6 is complete. The train

In [47]:
# one vs all approach, linear kernel
start = time.time()
   
preds = np.zeros((Xtest.shape[0], 10))

for i in range(10):
    # Train class i vs rest
    clf = SVM(kernel = "linear", iterations = 1, C=0.003)
    clf.fit(normalized_Xtrain, ytrain_ohe[:,i])
    preds[:, i] = clf.predict(normalized_Xtest, prob=True) # labels is going to be used for prediction on test data
    pred_labels = clf.predict(normalized_Xtrain) 
    pred_labels[pred_labels<1e-5] = -1
    pred_labels[pred_labels>=1e-5] = 1 # pred_labels are the labels predicted on training data
    # compute training accuracy
    score = accuracy(ytrain_ohe[:,i], pred_labels)
    print("Training class {} vs all is complete. The training accuracy is {:.2f}%".format(i, score*100))

ypred = np.argmax(preds, axis=1)

end = time.time()

score = accuracy(ytest, ypred)
print("The accuracy of multiclass classification is {:.2f}%".format(score*100))
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs all is complete. The training accuracy is 99.40%
Training class 1 vs all is complete. The training accuracy is 99.55%
Training class 2 vs all is complete. The training accuracy is 98.65%
Training class 3 vs all is complete. The training accuracy is 97.75%
Training class 4 vs all is complete. The training accuracy is 98.70%
Training class 5 vs all is complete. The training accuracy is 98.05%
Training class 6 vs all is complete. The training accuracy is 98.90%
Training class 7 vs all is complete. The training accuracy is 98.40%
Training class 8 vs all is complete. The training accuracy is 96.65%
Training class 9 vs all is complete. The training accuracy is 97.40%
The accuracy of multiclass classification is 90.20%
Takes 142.10 seconds.


In [48]:
# one vs all approach, poly kernel
start = time.time()
   
preds = np.zeros((Xtest.shape[0], 10))

for i in range(10):
    # Train class i vs rest
    clf = SVM(iterations=5, kernel = "poly", C=1)
    clf.fit(normalized_Xtrain, ytrain_ohe[:,i])
    preds[:, i] = clf.predict(normalized_Xtest, prob=True) # labels is going to be used for prediction on test data
    pred_labels = clf.predict(normalized_Xtrain) 
    pred_labels[pred_labels<1e-5] = -1
    pred_labels[pred_labels>=1e-5] = 1 # pred_labels are the labels predicted on training data
    # compute training accuracy
    score = accuracy(ytrain_ohe[:,i], pred_labels)
    print("Training class {} vs all is complete. The training accuracy is {:.2f}%".format(i, score*100))

ypred = np.argmax(preds, axis=1)

end = time.time()

score = accuracy(ytest, ypred)
print("The accuracy of multiclass classification is {:.2f}%".format(score*100))
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs all is complete. The training accuracy is 97.00%
Training class 1 vs all is complete. The training accuracy is 95.15%
Training class 2 vs all is complete. The training accuracy is 95.05%
Training class 3 vs all is complete. The training accuracy is 93.85%
Training class 4 vs all is complete. The training accuracy is 92.25%
Training class 5 vs all is complete. The training accuracy is 94.00%
Training class 6 vs all is complete. The training accuracy is 95.80%
Training class 7 vs all is complete. The training accuracy is 94.95%
Training class 8 vs all is complete. The training accuracy is 92.90%
Training class 9 vs all is complete. The training accuracy is 92.10%
The accuracy of multiclass classification is 88.20%
Takes 99.87 seconds.


In [49]:
# one vs all approach, rbf kernel
start = time.time()
   
preds = np.zeros((Xtest.shape[0], 10))

for i in range(10):
    # Train class i vs rest
    clf = SVM(iterations=5, kernel = "rbf", C=0.15)
    clf.fit(normalized_Xtrain, ytrain_ohe[:,i])
    preds[:, i] = clf.predict(normalized_Xtest, prob=True) # labels is going to be used for prediction on test data
    pred_labels = clf.predict(normalized_Xtrain) 
    pred_labels[pred_labels<1e-5] = -1
    pred_labels[pred_labels>=1e-5] = 1 # pred_labels are the labels predicted on training data
    # compute training accuracy
    score = accuracy(ytrain_ohe[:,i], pred_labels)
    print("Training class {} vs all is complete. The training accuracy is {:.2f}%".format(i, score*100))

ypred = np.argmax(preds, axis=1)

end = time.time()

score = accuracy(ytest, ypred)
print("The accuracy of multiclass classification is {:.2f}%".format(score*100))
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs all is complete. The training accuracy is 97.35%
Training class 1 vs all is complete. The training accuracy is 99.10%
Training class 2 vs all is complete. The training accuracy is 92.70%
Training class 3 vs all is complete. The training accuracy is 92.80%
Training class 4 vs all is complete. The training accuracy is 92.00%
Training class 5 vs all is complete. The training accuracy is 92.30%
Training class 6 vs all is complete. The training accuracy is 95.30%
Training class 7 vs all is complete. The training accuracy is 94.45%
Training class 8 vs all is complete. The training accuracy is 91.45%
Training class 9 vs all is complete. The training accuracy is 90.80%
The accuracy of multiclass classification is 85.40%
Takes 287.57 seconds.


In [50]:
# one vs all approach, sigmoid kernel
start = time.time()
   
preds = np.zeros((Xtest.shape[0], 10))

for i in range(10):
    # Train class i vs rest
    clf = SVM(iterations=5, kernel = "sigmoid", C=0.3)
    clf.fit(normalized_Xtrain, ytrain_ohe[:,i])
    preds[:, i] = clf.predict(normalized_Xtest, prob=True) # labels is going to be used for prediction on test data
    pred_labels = clf.predict(normalized_Xtrain) 
    pred_labels[pred_labels<1e-5] = -1
    pred_labels[pred_labels>=1e-5] = 1 # pred_labels are the labels predicted on training data
    # compute training accuracy
    score = accuracy(ytrain_ohe[:,i], pred_labels)
    print("Training class {} vs all is complete. The training accuracy is {:.2f}%".format(i, score*100))

ypred = np.argmax(preds, axis=1)

end = time.time()

score = accuracy(ytest, ypred)
print("The accuracy of multiclass classification is {:.2f}%".format(score*100))
print("Takes {:.2f} seconds.".format(end - start))

Training class 0 vs all is complete. The training accuracy is 98.15%
Training class 1 vs all is complete. The training accuracy is 98.95%
Training class 2 vs all is complete. The training accuracy is 95.65%
Training class 3 vs all is complete. The training accuracy is 95.80%
Training class 4 vs all is complete. The training accuracy is 96.20%
Training class 5 vs all is complete. The training accuracy is 94.20%
Training class 6 vs all is complete. The training accuracy is 96.75%
Training class 7 vs all is complete. The training accuracy is 97.25%
Training class 8 vs all is complete. The training accuracy is 93.05%
Training class 9 vs all is complete. The training accuracy is 91.35%
The accuracy of multiclass classification is 87.40%
Takes 217.92 seconds.
